In [1]:
from tensorflow.keras import backend as K 
import tensorflow as tf
from focal_loss import BinaryFocalLoss 

In [2]:
def iou_score(y_pred, y_true):
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.cast(y_pred, tf.float32)
  y_true = tf.reshape(y_true, [-1])
  y_pred = tf.reshape(y_pred, [-1])
  intersection = tf.reduce_sum(y_true * y_pred)
  score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
  return score

def dice_coef(y_test, y_pred):
    smooth = 1.
    
    y_true_f = K.flatten(y_test)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [3]:
def conv_block(x, filter_size, size):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    return conv

In [4]:
def repeat_elem(tensor, rep):
    # lambda function to repeat the elements of a tensor along an axis by a factor of rep.
    # If tensor has shape(None, 256,256,3),lambda will return a tensor of shape(None, 256,256,6), if specified axis=3 and rep=2.

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)


In [5]:
def gating_signal(input, out_size):
    
#resize the down layer feature map into the same dimension as the up layer feature map using 1x1 conv
   
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x


In [6]:
def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn


In [7]:
from tensorflow.keras import models, layers

In [8]:
def Attention_UNet(input_shape, NUM_CLASSES=1):

    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    
    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = conv_block(inputs, FILTER_SIZE, FILTER_NUM)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=3)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=3)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=3)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=3)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM)

    # 1*1 convolutional layers
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel
    
    # Model integration
    model = models.Model(inputs, conv_final)
    model.compile(optimizer='adam', loss=BinaryFocalLoss(gamma=3), metrics=['accuracy',dice_coef,iou_score])
    model.summary()

    return model


In [9]:
from keras.utils import normalize
import os
from tqdm import tqdm
import cv2
from PIL import Image
import random
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger,TensorBoard

In [10]:
#loading image paths
image_directory = 'C:/Users/PRATIK PAL/Desktop/New folder/CVC1/images/'
mask_directory = 'C:/Users/PRATIK PAL/Desktop/New folder/CVC1/masks/'

SIZE = 256
image_dataset = []    
mask_dataset = [] 

In [11]:
#reading the images, resize them and stored in a list
images = os.listdir(image_directory)
for i, image_name in tqdm(enumerate(images),total=len(images)):    
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))

#reading the masks, resize them and stored in a list
masks = os.listdir(mask_directory)
for i, image_name in tqdm(enumerate(masks),total=len(masks)):
    if (image_name.split('.')[1] == 'png'):
        mask = cv2.imread(mask_directory+image_name, 0)
        mask = Image.fromarray(mask)
        mask = mask.resize((SIZE, SIZE))
        mask_dataset.append(np.array(mask))

#Normalize images
image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis=1),3)
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.


100%|██████████| 470/470 [00:06<00:00, 76.03it/s]


In [12]:
#splitting them into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.3, random_state = 0)

In [13]:
#extracting height,width and no of channels to load the model
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)

In [14]:
callback = [
    ModelCheckpoint("cvcaumodel.h5"), CSVLogger('cvcauepoch.csv'),
    TensorBoard(),EarlyStopping(monitor='val_accuracy',patience=10,verbose=1,restore_best_weights=True)]

In [15]:
from keras.models import load_model

#AUnet model
au_model=Attention_UNet(input_shape)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

 rmalization)                   )                                                                 
                                                                                                  
 activation_8 (Activation)      (None, 16, 16, 1024  0           ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 16, 1024  9438208     ['activation_8[0][0]']           
                                )                                                                 
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 16, 16, 1024  4096       ['conv2d_9[0][0]']               
 rmalization)                   )                                                                 
          

 add_1 (Add)                    (None, 32, 32, 256)  0           ['conv2d_transpose_1[0][0]',     
                                                                  'conv2d_18[0][0]']              
                                                                                                  
 activation_16 (Activation)     (None, 32, 32, 256)  0           ['add_1[0][0]']                  
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 1)    257         ['activation_16[0][0]']          
                                                                                                  
 activation_17 (Activation)     (None, 32, 32, 1)    0           ['conv2d_20[0][0]']              
                                                                                                  
 up_sampling2d_2 (UpSampling2D)  (None, 64, 64, 1)   0           ['activation_17[0][0]']          
          

 conv2d_29 (Conv2D)             (None, 128, 128, 12  442496      ['concatenate_2[0][0]']          
                                8)                                                                
                                                                                                  
 batch_normalization_20 (BatchN  (None, 128, 128, 12  512        ['conv2d_29[0][0]']              
 ormalization)                  8)                                                                
                                                                                                  
 activation_23 (Activation)     (None, 128, 128, 12  0           ['batch_normalization_20[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_30 (Conv2D)             (None, 128, 128, 12  147584      ['activation_23[0][0]']          
          

                                                                                                  
 conv2d_38 (Conv2D)             (None, 256, 256, 1)  65          ['activation_29[0][0]']          
                                                                                                  
 batch_normalization_26 (BatchN  (None, 256, 256, 1)  4          ['conv2d_38[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_30 (Activation)     (None, 256, 256, 1)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
Total params: 37,333,513
Trainable params: 37,317,895
Non-trainable params: 15,618
__________________________________________________________________________________________________


In [16]:
#training the data
aunet_history = au_model.fit(X_train,y_train,batch_size=16,epochs=20,steps_per_epoch=25,
             validation_data=(X_test, y_test),shuffle=False,callbacks=callback)

Epoch 1/20
 1/25 [>.............................] - ETA: 46:15 - loss: 0.2180 - accuracy: 0.4279 - dice_coef: 0.1194 - iou_score: 0.0635

KeyboardInterrupt: 